In [1]:
import librosa
import numpy as np
import pandas as pd
import speech_recognition as sr 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Conv2D,MaxPooling2D
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler

2023-05-14 22:50:31.325591: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 22:50:33.285277: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-14 22:50:33.454115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/lib:/usr/local/lib64:/usr/lib64
2023-05-14 22:50:33.454220: I tensorflow/str

In [2]:
wav1, sample_rate1 = librosa.load("./train_data/audio_class_1.wav")
wav2, sample_rate2 = librosa.load("./train_data/audio_class_2.wav")

Feature extraction: Extract relevant features from the audio data using the librosa library. Some of the common features that can be extracted from audio data include Mel-Frequency Cepstral Coefficients (MFCCs), Spectral Centroid, Spectral Rolloff, Spectral Bandwidth, Chroma Frequencies, and Spectral Contrast.

In [4]:
hop_size = 15 #ms
FFT_size = 2048
def extract_features(audio, FFT_size=2048, hop_size=10, sample_rate=22050):
  audio = np.pad(audio, int(FFT_size / 2), mode='reflect')
  frame_len = np.round(sample_rate * hop_size / 1000).astype(int)
  frame_num = int((len(audio) - FFT_size) / frame_len) + 1
  frames = np.zeros((frame_num,FFT_size))
  
  for n in range(frame_num):
    frames[n] = audio[n*frame_len:n*frame_len+FFT_size]
  
  return frames

In [5]:
audio_frame1 = extract_features(wav1,FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate1)
audio_frame2 = extract_features(wav2,FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate2)
# audio_frame3 = extract_features(wav3,FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate3)

print("Framed audio shape for 1st audio : {}".format(audio_frame1.shape))
print("Framed audio shape for 2nd audio : {}".format(audio_frame2.shape))
# print("Framed audio shape for 3rd audio : {}".format(audio_frame3.shape))
# 

y_train1 = np.full((audio_frame1.shape[0]), 0)
y_train2 = np.full((audio_frame2.shape[0]), 1)
# y_train3 = np.full((audio_frame3.shape[0]), 0)

X_train = np.concatenate([audio_frame1,audio_frame2],axis=0)

y_train = np.concatenate([y_train1,y_train2],axis=0)

y_train = y_train.flatten()

y_train=pd.DataFrame(y_train)
y_train=pd.get_dummies(y_train[0:][0])

Framed audio shape for 1st audio : (11990, 2048)
Framed audio shape for 2nd audio : (11990, 2048)


In [6]:
y_train

,0,1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
23975,0,1
23976,0,1
23977,0,1
23978,0,1


# train data

# Model training

In [7]:
# neural netowrk eith 2048 features
def NN():
    model = Sequential()
    model.add(Dense(2048, input_shape=(2048,), activation = 'relu'))
    # model.add(Dropout(0.1))
    model.add(Dense(1028, activation = 'relu'))
    model.add(Dropout(0.25))  
    model.add(Dense(248, activation = 'relu'))
    model.add(Dropout(0.5))    
    model.add(Dense(2, activation = 'sigmoid'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')
    return model

In [8]:
model = NN()
model.fit(X_train,y_train,epochs = 5)

2023-05-14 22:51:13.720651: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-14 22:51:13.723149: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/lib:/usr/local/lib64:/usr/lib64
2023-05-14 22:51:13.723996: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/lib:/usr/local/lib64:/usr/lib64
2023-05-14 22:51:13.724695: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open 

Epoch 1/5
750/750 [==============================] - 60s 74ms/step - loss: 0.7007 - accuracy: 0.5152
Epoch 2/5
750/750 [==============================] - 78s 104ms/step - loss: 0.6806 - accuracy: 0.5732
Epoch 3/5
750/750 [==============================] - 65s 86ms/step - loss: 0.6295 - accuracy: 0.6354
Epoch 4/5
750/750 [==============================] - 63s 83ms/step - loss: 0.5332 - accuracy: 0.7131
Epoch 5/5
750/750 [==============================] - 62s 82ms/step - loss: 0.4416 - accuracy: 0.7755


In [9]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# # Evaluate the trained model on the testing data
# def evaluate_model(clf, X_test, y_test):
#     y_pred = clf.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred, average='weighted')
#     recall = recall_score(y_test, y_pred, average='weighted')
#     f1 = f1_score(y_test, y_pred, average='weighted')
#     return accuracy, precision, recall, f1


In [10]:
wav3, sample_rate3 = librosa.load("./test_data/test_audio.wav")
x_test = extract_features(wav3, FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate3)
y_predicted = model.predict(x_test)
y_predicted_labels = [np.argmax(i) for i in y_predicted]

188/188 [==============================] - 6s 30ms/step


In [11]:
y_predicted_labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [12]:
# text to speech using google api
r = sr.Recognizer()
filename='./test_data/test_audio.wav'
frame_length=11990/sample_rate3
j=0
i=0
for j in range(len(y_predicted_labels)):
    if y_predicted_labels[j]==0:
        print('Speaker 1:\t')
        k=0
        while y_predicted_labels[j]==0:
            k=k+1
            if j+k> len(y_predicted_labels):
                break
        with sr.AudioFile(filename) as source:
            audio_data=r.record(source,duration=frame_length*k,offset=frame_length*j)
            text=r.recognize_google(audio_data)
            print(text)
            j=j+k
    elif y_predicted_labels[j]==1:
        print('Speaker 2:\t')
        k=0
        while y_predicted_labels[j]==1 :
            k=k+1
            if j+k> len(y_predicted_labels):
                break
        try:
            with sr.AudioFile(filename) as source:
                audio_data=r.record(source,duration=frame_length*k,offset=frame_length*j)
                text=r.recognize_google(audio_data)
                print(text)
                j=j+k

        except:
            j=j+k
    i=j
        

Speaker 1:	
Maharashtra cases and with this India stallion covid-19 virus induced infection has caused Holi restrictions for religious and political reasons
Speaker 1:	
one day trip places and with this India Alia with covid-19 virus induced infection is caused Holi
Speaker 1:	
cases and with this India stereo with covid-19 virus induced infection has caused Holi religious and political reasons
Speaker 1:	
